## Memo
- 設備
  - laptop
  - RTX 3060 Laptop 6G
- 可以做的改進
  - Black-grass、Loose Silky-bent這兩類一直預測相互有錯，實際上肉眼也看不太出來，可以嘗試對這兩類做更多處理。
  - 類別不平衡我用了cutmix處理，或許可以調weight來解決，不知哪個效果好。
  - 調參，但跑一次fold要10小時，設備不足所以我懶得繼續調了。

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image

from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision import models
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tqdm.auto import tqdm

In [4]:
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
!nvidia-smi

Sat Jul 29 17:32:03 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 L...  WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   48C    P8               15W /  N/A|    498MiB /  6144MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Data preprocessing

In [6]:
root_path='../plant-seedlings-classification/'

In [7]:
clas=os.listdir(root_path+'train/')
label=sorted(list(set(clas)))
class_to_num=dict(zip(label,range(len(clas))))
print(class_to_num,'\n')

num_to_class = {v : k for k, v in class_to_num.items()}
print(num_to_class)

{'Black-grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common Chickweed': 3, 'Common wheat': 4, 'Fat Hen': 5, 'Loose Silky-bent': 6, 'Maize': 7, 'Scentless Mayweed': 8, 'Shepherds Purse': 9, 'Small-flowered Cranesbill': 10, 'Sugar beet': 11} 

{0: 'Black-grass', 1: 'Charlock', 2: 'Cleavers', 3: 'Common Chickweed', 4: 'Common wheat', 5: 'Fat Hen', 6: 'Loose Silky-bent', 7: 'Maize', 8: 'Scentless Mayweed', 9: 'Shepherds Purse', 10: 'Small-flowered Cranesbill', 11: 'Sugar beet'}


In [8]:
imgs=[]
labels=[]
for i in clas:
    for j in range(len(os.listdir(root_path+'train/'+ i))):
        imgs.append(root_path+'train/'+ i +'/'+os.listdir(root_path+'train/'+ i)[j])
        labels.append(i)
        
df=pd.DataFrame({"imgs":imgs,"labels":labels})
df["labels"].value_counts()

##類別數量不平衡

Loose Silky-bent             654
Common Chickweed             611
Scentless Mayweed            516
Small-flowered Cranesbill    496
Fat Hen                      475
Charlock                     390
Sugar beet                   385
Cleavers                     287
Black-grass                  263
Shepherds Purse              231
Common wheat                 221
Maize                        221
Name: labels, dtype: int64

In [9]:
class train_valid_dataset(Dataset):
    def __init__(self,file_path,tfm=None):
#         print(file_path)
        self.file_path=file_path
        self.transform=tfm
        self.imgs=list(df["imgs"])
        self.labels=list(df["labels"])

#         print(len(self.imgs)==len(self.labels)) 
        
    def __getitem__(self,idx):
        img_as_img=Image.open(self.imgs[idx]).convert('RGB')
        img_as_img=self.transform(img_as_img)

        label=self.labels[idx]
        label=class_to_num[label]
        
        return img_as_img,label
    
    def __len__(self):   
        return len(self.imgs)

In [10]:
class test_dataset(Dataset):
    def __init__(self,file_path,tfm=None):
        self.file_path=file_path
        self.transform=tfm
        self.imgs=[]
        for i in range(len(os.listdir(self.file_path))):
            self.imgs.append(self.file_path+os.listdir(self.file_path)[i])
        
    def __getitem__(self,idx):
        img_as_img=Image.open(self.imgs[idx]).convert('RGB')
        img_as_img=self.transform(img_as_img)
        
        return img_as_img,self.imgs[idx]
    
    def __len__(self):   
        return len(self.imgs)

## Model

In [11]:
#原model參數用imagenet練出來的，dataset差有點多，參數跟著調結果好一點?
def set_parameter_requires_grad(model, feature_extracting):

    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False
            
def resnet_model(num_classes, feature_extract = False):
    model_ft = models.resnet50(pretrained=True)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

model=resnet_model(12)

## Config

In [12]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)
results = {}

n_epochs=60
patience=30
batch_size=32

#Adam
learning_rate=2e-4
weight_decay=1e-4

#SGD
# learning_rate=1e-1
# weight_decay=1e-2

In [13]:
train_loss_function=CutMixCrossEntropyLoss(True)
valid_loss_function=nn.CrossEntropyLoss()

In [14]:
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)
# optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate,weight_decay=weight_decay)

In [15]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=10)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=16, T_mult=1)

## Data Augumentation , Dataset and DataLoader

In [16]:
seed=1234
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [17]:
# def train_valid_split(data_set, valid_ratio, seed):
#     valid_set_size = int(valid_ratio * len(data_set)) 
#     train_set_size = len(data_set) - valid_set_size
#     train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
#     return train_set, valid_set

In [18]:
train_transform=transforms.Compose([
    transforms.Resize((299,299)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(180),
    transforms.RandomAffine(degrees=10, translate=(0.25, 0.25), scale=(0.9, 1.1)),
    transforms.ColorJitter(brightness=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

test_transform=transforms.Compose([
    transforms.Resize((299,299)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])


In [19]:
train_val_dataset=train_valid_dataset(root_path+'train/',tfm=None)
# train_dataset,valid_dataset=train_valid_split(dataset,0.2,seed)
test_datase=test_dataset(root_path+'test/',tfm=test_transform)

test_dataloader=DataLoader(test_datase, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [175]:
# for batch in train_dataset:
#     images, labels = batch
#     images=images.permute(1,2,0).numpy()
#     plt.imshow(images)
#     plt.show()

## K-fold Training

In [ ]:
train_loss_epoch=[[] for _ in range(k_folds)]
valid_loss_epoch=[[] for _ in range(k_folds)]
valid_accs_epoch=[[] for _ in range(k_folds)]
epoch_count=[[] for _ in range(k_folds)]
for fold, (train,valid) in enumerate(kfold.split(train_val_dataset)):
    best_acc = 0
    stale = 0
    counter=0
    print(f'FOLD {fold}')
    print('--------------------------------------')
    
    train_subsampler=torch.utils.data.SubsetRandomSampler(train)
    valid_subsampler=torch.utils.data.SubsetRandomSampler(valid)

    train_dataloader=DataLoader(CutMix(train_valid_dataset(root_path+'train/',tfm=train_transform), num_class=12, beta=1.0, prob=0.5, num_mix=2)
                            , batch_size=batch_size, sampler=train_subsampler, num_workers=0)

    valid_dataloader=DataLoader(train_valid_dataset(root_path+'train/',tfm=test_transform), batch_size=batch_size, sampler=valid_subsampler, num_workers=0)
    
    for epoch in range(0,n_epochs):
        counter+=1
        model=model.to(device)
        model.train()
        train_loss=[]
    #     train_accs=[]
        for batch in tqdm(train_dataloader):
            imgs,label=batch
    #         print(imgs.shape,label.shape)
            imgs=imgs.to(device)
            label=label.to(device)

            logits=model(imgs)

            loss=train_loss_function(logits,label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    #         acc = (logits.argmax(dim=-1) == label.to(device)).float().mean()

            train_loss.append(loss.item())
    #         train_accs.append(acc)

        scheduler.step()

        train_loss=sum(train_loss)/len(train_loss)
    #     train_accs = sum(train_accs) / len(train_accs)

        train_loss_epoch[fold].append(train_loss)
    #     train_accs_epoch.append(train_accs)
 
        print(f"[ Train | {epoch+1}/{n_epochs} ] loss = {train_loss:.5f}")

        print('Starting validation')   

        model.eval()

        valid_loss=[]
        valid_accs=[]

        for batch in tqdm(valid_dataloader):
            imgs,label=batch

            imgs=imgs.to(device)
            label=label.to(device)

            with torch.no_grad():
                logits=model(imgs)

            loss=valid_loss_function(logits,label)

            acc = (logits.argmax(dim=-1) == label.to(device)).float().mean()

            valid_loss.append(loss.item())
            valid_accs.append(acc)

        valid_loss=sum(valid_loss)/len(valid_loss)
        valid_accs = sum(valid_accs) / len(valid_accs)
        valid_loss_epoch[fold].append(valid_loss)
        valid_accs_epoch[fold].append(valid_accs)

        print(f"[ Valid | {epoch+1}/{n_epochs} ] loss = {valid_loss:.5f}, acc = {valid_accs:.5f}")
        print('--------------------------------------')
        epoch_count[fold].append(counter)
        results[fold] = valid_accs
        
        # save models
        if valid_accs > best_acc:
            print(f"Best model found at epoch {epoch+1}, saving model")
            torch.save(model.state_dict(), f"best-{fold}.ckpt") # only save best to prevent output memory exceed error
            best_acc = valid_accs
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break 

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} ')
    total_summation += value
    
print(f'Average: {total_summation/len(results.items())}')

In [ ]:
for fold in range(k_folds):
    epochs=np.arange(0,max(epoch_count[fold]),1)
    
    if isinstance(valid_accs_epoch[fold][0],np.ndarray):
        pass
    else:
        for i in range(len(valid_accs_epoch[fold])):
            valid_accs_epoch[fold][i]=valid_accs_epoch[fold][i].cpu().numpy()


    plt.plot(epochs, train_loss_epoch[fold],'-', label='Training Loss')
    plt.plot(epochs, valid_loss_epoch[fold],'m--', label='Validation Loss')
#     plt.plot(epochs, train_accs_epoch[fold],'g-.', label='Training acc')
    plt.plot(epochs, valid_accs_epoch[fold], 'r:',label='Validation acc')

    plt.xlabel('Epochs')
    plt.title('Training and Validation Loss/accs Curves'+'\n'+"   "+f"valid_accs:{max(valid_accs_epoch[fold]):.2f}"+'\n'+f"train_loss:{min(train_loss_epoch[fold]):.2f}"+"   "+f"valid_loss:{min(valid_loss_epoch[fold]):.2f}")

    plt.legend(['train loss', 'valid loss','train acc' ,'valid acc'])
    plt.grid(True)
    plt.savefig(f'train_valid_{fold}.png')
    plt.show()

## wrong predict in valid loss

In [ ]:
for fold in range(k_folds):
    best_model=resnet_model(12).to(device)
    best_model.load_state_dict(torch.load(f'best-{fold}.ckpt'))
    best_model.eval()
    label=[]
    prediction = []
    with torch.no_grad():
        for imgs,lab in tqdm(valid_dataloader):
            imgs=imgs.to(device)
            label_p=best_model(imgs)
            label_predict = np.argmax(label_p.cpu().data.numpy(), axis=1)
            label+=lab.cpu().squeeze().tolist()
            prediction+=label_predict.squeeze().tolist()

    # print(label)
    # print(prediction)
    cm = confusion_matrix(label, prediction)


    # 绘制混淆矩阵
    print(class_to_num)
    plt.figure(figsize=(15, 15))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.savefig(f'confusion_matrix{fold}.png')
    plt.show()

## Testing 

In [20]:
for test_fold in range(k_folds):
    predict_path = f'submission-fold-{test_fold}.csv'
    best_model=resnet_model(12).to(device)
    best_model.load_state_dict(torch.load(f'best-{test_fold}.ckpt'))
    best_model.eval()
    img_name=[]
    prediction = []
    with torch.no_grad():
        for imgs,img_n in tqdm(test_dataloader):
            imgs=imgs.to(device)
            label_p=best_model(imgs)
            label_predict = np.argmax(label_p.cpu().data.numpy(), axis=1)
            img_name.append(img_n)
            prediction+=label_predict.squeeze().tolist()

    name_flatten=[element for sublist in img_name for element in sublist]
    name_flatten=[item.split('/')[3] for item in name_flatten]

    preds = []
    for i in prediction:
        preds.append(num_to_class[i])

    df=pd.DataFrame({"file":name_flatten,"species":preds})
    df.to_csv(predict_path,index = False)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [21]:
df0 = pd.read_csv('submission-fold-0.csv')
df1 = pd.read_csv('submission-fold-1.csv')
df2 = pd.read_csv('submission-fold-2.csv')
df3 = pd.read_csv('submission-fold-3.csv')
df4 = pd.read_csv('submission-fold-4.csv')

In [22]:
list_num_label0 = []
for i in df0['species']:
    list_num_label0.append(class_to_num[i])
df0['num_label0']=list_num_label0
df0.head()

,file,species,num_label0
0,0021e90e4.png,Small-flowered Cranesbill,10
1,003d61042.png,Fat Hen,5
2,007b3da8b.png,Sugar beet,11
3,0086a6340.png,Common Chickweed,3
4,00c47e980.png,Sugar beet,11


In [23]:
list_num_label1 = []
for i in df1['species']:
    list_num_label1.append(class_to_num[i])
df1['num_label1']=list_num_label1
df1.head()

,file,species,num_label1
0,0021e90e4.png,Small-flowered Cranesbill,10
1,003d61042.png,Fat Hen,5
2,007b3da8b.png,Sugar beet,11
3,0086a6340.png,Common Chickweed,3
4,00c47e980.png,Sugar beet,11


In [24]:
list_num_label2 = []
for i in df2['species']:
    list_num_label2.append(class_to_num[i])
df2['num_label2']=list_num_label2
df2.head()

,file,species,num_label2
0,0021e90e4.png,Small-flowered Cranesbill,10
1,003d61042.png,Fat Hen,5
2,007b3da8b.png,Sugar beet,11
3,0086a6340.png,Common Chickweed,3
4,00c47e980.png,Sugar beet,11


In [25]:
list_num_label3 = []
for i in df3['species']:
    list_num_label3.append(class_to_num[i])
df3['num_label3']=list_num_label3
df3.head()

,file,species,num_label3
0,0021e90e4.png,Small-flowered Cranesbill,10
1,003d61042.png,Fat Hen,5
2,007b3da8b.png,Sugar beet,11
3,0086a6340.png,Common Chickweed,3
4,00c47e980.png,Sugar beet,11


In [26]:
list_num_label4 = []
for i in df4['species']:
    list_num_label4.append(class_to_num[i])
df4['num_label4']=list_num_label4
df4.head()

,file,species,num_label4
0,0021e90e4.png,Small-flowered Cranesbill,10
1,003d61042.png,Fat Hen,5
2,007b3da8b.png,Sugar beet,11
3,0086a6340.png,Common Chickweed,3
4,00c47e980.png,Sugar beet,11


In [27]:
df_all = df0.copy()
df_all.drop(['species'],axis=1,inplace=True)
df_all.head()

,file,num_label0
0,0021e90e4.png,10
1,003d61042.png,5
2,007b3da8b.png,11
3,0086a6340.png,3
4,00c47e980.png,11


In [28]:
df_all['num_label1']=list_num_label1
df_all['num_label2']=list_num_label2
df_all['num_label3']=list_num_label3
df_all['num_label4']=list_num_label4
df_all.head()

,file,num_label0,num_label1,num_label2,num_label3,num_label4
0,0021e90e4.png,10,10,10,10,10
1,003d61042.png,5,5,5,5,5
2,007b3da8b.png,11,11,11,11,11
3,0086a6340.png,3,3,3,3,3
4,00c47e980.png,11,11,11,11,11


In [29]:
df_all_transpose = df_all.copy().drop(['file'],axis=1).transpose()
df_all_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,784,785,786,787,788,789,790,791,792,793
num_label0,10,5,11,3,11,6,3,5,6,6,...,6,0,8,8,9,6,11,11,1,4
num_label1,10,5,11,3,11,6,3,5,6,6,...,6,0,8,8,9,6,11,11,1,4
num_label2,10,5,11,3,11,6,3,5,6,6,...,6,0,8,8,9,6,11,11,1,4
num_label3,10,5,11,3,11,6,3,5,6,6,...,6,0,8,8,9,6,11,11,1,4
num_label4,10,5,11,3,11,6,3,5,6,6,...,6,0,8,8,9,6,11,11,1,4


In [30]:
df_mode = df_all_transpose.mode().transpose()
df_mode.head()

,0
0,10
1,5
2,11
3,3
4,11


In [31]:
voting_class = []
for each in df_mode[0]:
    voting_class.append(num_to_class[each])
voting_class

['Small-flowered Cranesbill',
 'Fat Hen',
 'Sugar beet',
 'Common Chickweed',
 'Sugar beet',
 'Loose Silky-bent',
 'Common Chickweed',
 'Fat Hen',
 'Loose Silky-bent',
 'Loose Silky-bent',
 'Fat Hen',
 'Small-flowered Cranesbill',
 'Sugar beet',
 'Scentless Mayweed',
 'Sugar beet',
 'Fat Hen',
 'Scentless Mayweed',
 'Scentless Mayweed',
 'Common Chickweed',
 'Shepherds Purse',
 'Common Chickweed',
 'Small-flowered Cranesbill',
 'Shepherds Purse',
 'Sugar beet',
 'Sugar beet',
 'Maize',
 'Scentless Mayweed',
 'Scentless Mayweed',
 'Common Chickweed',
 'Sugar beet',
 'Common Chickweed',
 'Shepherds Purse',
 'Black-grass',
 'Sugar beet',
 'Loose Silky-bent',
 'Black-grass',
 'Loose Silky-bent',
 'Loose Silky-bent',
 'Common Chickweed',
 'Maize',
 'Loose Silky-bent',
 'Common wheat',
 'Common Chickweed',
 'Small-flowered Cranesbill',
 'Sugar beet',
 'Sugar beet',
 'Maize',
 'Scentless Mayweed',
 'Maize',
 'Small-flowered Cranesbill',
 'Loose Silky-bent',
 'Sugar beet',
 'Cleavers',
 'Loose

In [32]:
df_all['species'] = voting_class
df_all.head()

,file,num_label0,num_label1,num_label2,num_label3,num_label4,species
0,0021e90e4.png,10,10,10,10,10,Small-flowered Cranesbill
1,003d61042.png,5,5,5,5,5,Fat Hen
2,007b3da8b.png,11,11,11,11,11,Sugar beet
3,0086a6340.png,3,3,3,3,3,Common Chickweed
4,00c47e980.png,11,11,11,11,11,Sugar beet


In [33]:
df_submission = df_all[['file','species']].copy()
df_submission.head()

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet


In [34]:
df_submission.to_csv('./submission-resnet.csv', index=False)
print('Voting results of resnest successfully saved!')

Voting results of resnest successfully saved!
